In [1]:
import findspark
findspark.init()

import requests
import json
import unidecode
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

spark = SparkSession.builder \
      .master("local[1]") \
      .appName("SparkByExamples.com") \
      .getOrCreate()

22/09/04 21:14:00 WARN Utils: Your hostname, Javiers-MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 192.168.1.104 instead (on interface en0)
22/09/04 21:14:00 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/09/04 21:14:02 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
# Buscar cidades do Vale do Paraíba
# TODO
url = 'https://servicodados.ibge.gov.br/api/v1/localidades/mesorregioes/3513/municipios'
response = requests.get(url)

# Criar data frame com as cidades
# TODO
raw_df = spark.read.json(spark.sparkContext.parallelize([response.json()]), multiLine=True)
# raw_df.printSchema()
temp_df = (
        raw_df
        .select(
                'id', 
                'nome',
                col('regiao-imediata.nome').alias('regiao_imediata'),
                col('microrregiao.mesorregiao.UF.nome').alias('estado'))
        .withColumnRenamed('id', 'id_cidade')
        .withColumnRenamed('nome', 'cidade')
)
# Criar view com as cidades
# TODO
temp_df.createOrReplaceTempView('cidades')
cidades = spark.sql('SELECT * FROM cidades')
cidades.show()


+---------+------------------+--------------------+---------+
|id_cidade|            cidade|     regiao_imediata|   estado|
+---------+------------------+--------------------+---------+
|  3502507|         Aparecida|       Guaratinguetá|São Paulo|
|  3503158|            Arapeí|            Cruzeiro|São Paulo|
|  3503505|            Areias|            Cruzeiro|São Paulo|
|  3504909|           Bananal|            Cruzeiro|São Paulo|
|  3508504|          Caçapava| São José dos Campos|São Paulo|
|  3508603|Cachoeira Paulista|            Cruzeiro|São Paulo|
|  3509700|  Campos do Jordão|Taubaté - Pindamo...|São Paulo|
|  3509957|             Canas|       Guaratinguetá|São Paulo|
|  3510500|     Caraguatatuba|Caraguatatuba - U...|São Paulo|
|  3513405|          Cruzeiro|            Cruzeiro|São Paulo|
|  3513603|             Cunha|       Guaratinguetá|São Paulo|
|  3518404|     Guaratinguetá|       Guaratinguetá|São Paulo|
|  3520202|           Igaratá| São José dos Campos|São Paulo|
|  35204

In [9]:
# Buscar previsão do tempo para as cidades
# TODO
estado = 'SP'
api_key = 'api_key'
json_list = []
for cidade in cidades.collect():
    cidade = cidade.cidade
    cidade = cidade.replace(' ', '%20')
    url = f'https://api.hgbrasil.com/weather?key={api_key}&city_name={cidade},{estado}&array_limit=5'
    response = requests.get(url)
    json_list.append(response.json()['results'])

raw_df = spark.read.json(spark.sparkContext.parallelize(json_list), multiLine=True)
# raw_df.printSchema()

# Criar data frame com as previsões
# TODO
temp_df = (
            raw_df
            .select('city', 'temp', 'time', 'sunrise', 'sunset', 'wind_speedy',
            explode('forecast').alias('forecast'))
            .select('city', 'temp', 'time', 'sunrise', 'sunset', 'wind_speedy', 'forecast.*')
            .withColumn('pais', lit('Brasil'))
            .withColumn('city', 
            when(raw_df.city.endswith(', SP'),  regexp_replace(raw_df.city, ', SP', '')))
            .withColumn('Vai chover', when(col('condition') == 'rain', lit('Sim')).otherwise(lit('Não')))
)
# Criar view com as previsões
# TODO

temp_df.createOrReplaceTempView('previsoes')
previsoes = spark.sql('SELECT * FROM previsoes')
previsoes.show()

+---------+----+-----+--------+--------+-----------+-----------+-----+--------------------+---+---+-------+------+----------+
|     city|temp| time| sunrise|  sunset|wind_speedy|  condition| date|         description|max|min|weekday|  pais|Vai chover|
+---------+----+-----+--------+--------+-----------+-----------+-----+--------------------+---+---+-------+------+----------+
|Aparecida|  14|21:12|06:08 am|05:51 pm|  0.88 km/h|cloudly_day|04/09|       Tempo nublado| 17| 13|    Dom|Brasil|       Não|
|Aparecida|  14|21:12|06:08 am|05:51 pm|  0.88 km/h|       rain|05/09|     Chuvas esparsas| 26| 12|    Seg|Brasil|       Sim|
|Aparecida|  14|21:12|06:08 am|05:51 pm|  0.88 km/h|cloudly_day|06/09|       Tempo nublado| 25| 14|    Ter|Brasil|       Não|
|Aparecida|  14|21:12|06:08 am|05:51 pm|  0.88 km/h|       rain|07/09|               Chuva| 30| 13|    Qua|Brasil|       Sim|
|Aparecida|  14|21:12|06:08 am|05:51 pm|  0.88 km/h|       rain|08/09|     Chuvas esparsas| 28| 14|    Qui|Brasil|    

In [43]:
# Criar DF da Tabela 1
# TODO
df_join = spark.sql('SELECT * FROM previsoes JOIN cidades ON previsoes.city = cidades.cidade')
tabela1 = (
    df_join
    .select('id_cidade', 'cidade', 'estado', 'pais', 'temp', 'date', 'time', 'description', 'sunrise', 'sunset', 'wind_speedy', 'condition', 'Vai chover')
    .withColumnRenamed('date', 'data')
    .withColumnRenamed('time', 'hora')
    .withColumnRenamed('temp', 'temperatura')
    .withColumnRenamed('description', 'descricao')
    .withColumnRenamed('wind_speedy', 'velocidade_vento')
    .withColumnRenamed('condition', 'condicao')
    .withColumnRenamed('sunrise', 'nascer_sol')
    .withColumnRenamed('sunset', 'por_sol')
    )
tabela1.show()

+---------+---------+---------+------+-----------+-----+-----+--------------------+----------+--------+----------------+-----------+----------+
|id_cidade|   cidade|   estado|  pais|temperatura| data| hora|           descricao|nascer_sol| por_sol|velocidade_vento|   condicao|Vai chover|
+---------+---------+---------+------+-----------+-----+-----+--------------------+----------+--------+----------------+-----------+----------+
|  3502507|Aparecida|São Paulo|Brasil|         14|04/09|21:12|       Tempo nublado|  06:08 am|05:51 pm|       0.88 km/h|cloudly_day|       Não|
|  3502507|Aparecida|São Paulo|Brasil|         14|05/09|21:12|     Chuvas esparsas|  06:08 am|05:51 pm|       0.88 km/h|       rain|       Sim|
|  3502507|Aparecida|São Paulo|Brasil|         14|06/09|21:12|       Tempo nublado|  06:08 am|05:51 pm|       0.88 km/h|cloudly_day|       Não|
|  3502507|Aparecida|São Paulo|Brasil|         14|07/09|21:12|               Chuva|  06:08 am|05:51 pm|       0.88 km/h|       rain|    

In [45]:
# Criar DF da Tabela 2
# TODO
temp_tabela2 = (
    df_join
    .select('cidade', 'Vai chover', 'date')
    .groupBy('cidade', 'Vai chover')
    .agg(count('date').alias('Quantidade de dias de chuva'))
)
tabela2 = (
    temp_tabela2
    .select('cidade', 'Vai chover', 'Quantidade de dias de chuva')
    .groupBy('cidade')
    .pivot('Vai chover')
    .agg(first('Quantidade de dias de chuva'))
    .withColumnRenamed('Sim', 'Quantidade de dias de chuva')
    .withColumnRenamed('Não', 'Quantidade de dias com chuva')
    .withColumn('Quanntidade de dias mapeados', col('Quantidade de dias de chuva') + col('Quantidade de dias com chuva'))
)
tabela2.show()

+------------------+----------------------------+---------------------------+----------------------------+
|            cidade|Quantidade de dias com chuva|Quantidade de dias de chuva|Quanntidade de dias mapeados|
+------------------+----------------------------+---------------------------+----------------------------+
|         Aparecida|                           4|                          6|                          10|
|            Arapeí|                           3|                          7|                          10|
|            Areias|                           2|                          8|                          10|
|           Bananal|                           3|                          7|                          10|
|Cachoeira Paulista|                           4|                          6|                          10|
|  Campos do Jordão|                           2|                          8|                          10|
|             Canas|                 

In [46]:
# Exportar CSVs
# TODO
tabela1.write.csv('tabela1.csv')
tabela2.write.csv('tabela2.csv')